In [20]:
import tensorflow as tf
import tensorflow.contrib.keras as keras
import os
import struct
import numpy as np

In [21]:
#Carichiamo il database: ritorna due array n (numero di campioni) x m (numero di feature (pixel)).
#60000 training digits, e 10000 test digits. Le immagini sono 28x28 pixel in scala di grigio, che
#qui vengono convertite in vettori unidimensionali di riga (784 righe oper immagine). 
#Il secondo vettore contiene le variabili target (class labels 0-9 delle cifre scritte a mano)
def load_mnist(path, kind='train'):
    labels_path=os.path.join(path,
                            '%s-labels-idx1-ubyte' % kind)
    images_path=os.path.join(path,
                            '%s-images-idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
                            magic, n= struct.unpack('>II', 
                                                    lbpath.read(8))
                            labels= np.fromfile(lbpath, 
                                                dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
                            magic, num, rwos, cols= struct.unpack('>IIII', 
                                                                  imgpath.read(16))
                            images= np.fromfile(imgpath, 
                                                dtype=np.uint8).reshape(
                                                len(labels), 784)
                            images=((images/255.)-.5)*2
    return images, labels                       
                            

In [24]:
X_train, y_train= load_mnist('', kind='train')
print ('Rows: %d, Columns: %d' % (X_train.shape[0], X_train.shape[1]))


Rows: 60000, Columns: 784


In [25]:
X_test, y_test= load_mnist('', kind='t10k')
print ('Rows: %d, Columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 10000, Columns: 784


In [27]:
#Media e deviazione standard per standardizzazione
mean_vals=np.mean(X_train, axis=0)
std_val=np.std(X_train)
X_train_centered=(X_train - mean_vals)/std_val
X_test_centered=(X_test - mean_vals)/std_val
del X_train, X_test


In [28]:
print(X_train_centered.shape, y_train.shape)

(60000, 784) (60000,)


In [29]:
print(X_test_centered.shape, y_test.shape)

(10000, 784) (10000,)


In [30]:
#Settiamo il random seed
np.random.seed(123)
tf.set_random_seed(123)

In [31]:
#Codifica one-hot encoding
y_train_onehot=keras.utils.to_categorical(y_train)

In [32]:
print('Le prime 3 label: ', y_train[:3])

Le prime 3 label:  [5 0 4]


In [33]:
print('Le prime 3 label codificate one-hot: ', y_train_onehot[:3])

Le prime 3 label codificate one-hot:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [38]:
#Rete neurale
model=keras.models.Sequential()
model.add(
    keras.layers.Dense(
        units=50, #dimensione di uscita
        input_dim=X_train_centered.shape[1], # dimensione di ingresso
        kernel_initializer='glorot_uniform', # inizializzazioen dei pesi
        bias_initializer='zeros', #inizializzazione dei pesi
        activation='tanh'))# funzione di attivazione
model.add(
    keras.layers.Dense(
        units=50, #dimensione di uscita
        input_dim=50, # dimensione di ingresso
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))
model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1], #dimensione di uscita
        input_dim=50, # dimensione di ingresso
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))
#Ottimizzatore della funzione costo
sgd_optimizer = keras.optimizers.SGD(# minimizziamo con stochastic gradient descent
    lr=0.001, decay=1e-7, momentum=.9)
model.compile(optimizer=sgd_optimizer,
             loss='categorical_crossentropy') #funzione costo logistic regression 

In [40]:
#Fit
history= model.fit(X_train_centered, y_train_onehot,
                  batch_size=64, epochs=50, #apprendimento su minibatch di 64 campioni con 50 iterazioni
                  verbose=1,
                  validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 3s 61us/step - loss: 0.0426 - val_loss: 0.1057
Epoch 2/50
54000/54000 [==============================] - 3s 51us/step - loss: 0.0414 - val_loss: 0.1060
Epoch 3/50
54000/54000 [==============================] - 3s 51us/step - loss: 0.0405 - val_loss: 0.1064
Epoch 4/50
54000/54000 [==============================] - 3s 51us/step - loss: 0.0394 - val_loss: 0.1063
Epoch 5/50
54000/54000 [==============================] - 3s 53us/step - loss: 0.0384 - val_loss: 0.1057
Epoch 6/50
54000/54000 [==============================] - 3s 61us/step - loss: 0.0374 - val_loss: 0.1059
Epoch 7/50
54000/54000 [==============================] - 3s 61us/step - loss: 0.0364 - val_loss: 0.1065
Epoch 8/50
54000/54000 [==============================] - 3s 55us/step - loss: 0.0356 - val_loss: 0.1073
Epoch 9/50
54000/54000 [==============================] - 3s 56us/step - loss: 0.0347 - val_loss: 0.1074
Epoch 

In [42]:
#Predict
y_train_pred=model.predict_classes(X_train_centered, verbose=0)


In [43]:
correct_preds=np.sum(y_train == y_train_pred, axis=0)
train_acc=correct_preds/y_train.shape[0]
print('Prime 3 predizioni: ', y_train_pred[:3])
print('Accuratezza training: %.2f%%' % (train_acc*100))

Prime 3 predizioni:  [5 0 4]
Accuratezza training: 99.61%


In [44]:
y_test_pred=model.predict_classes(X_test_centered, verbose=0)
correct_preds=np.sum(y_test == y_test_pred, axis=0)
test_acc=correct_preds/y_test.shape[0]
print('Prime 3 predizioni: ', y_train_pred[:3])
print('Accuratezza test: %.2f%%' % (test_acc*100))

Prime 3 predizioni:  [5 0 4]
Accuratezza test: 96.27%
